## Jupyter Notebook zur Summierung der Levenshtein Werte pro Ort

Ein Ansatz, um räumliche Unterschiede herauszufinden ist, die Levenshtein Distanzen pro Ort zu summieren.
Daraus werden erneut Karten erstellt.

In [1]:
import pandas as pd

words = ["ameise", "begräbnis", "deichsel", "elster", "fledermaus", "gurke", "hagebutte", "hebamme", "kartoffel", "maulwurf", "pflaume", "stecknadel", "ziege", "zimmerfliege"]

In [2]:
df_gesamt = pd.DataFrame()
for word in words:
    df = pd.read_csv(f"/home/kopatsch/Masterarbeit/MA/masterarbeit/data/levenshtein_single_files/{word}.csv", sep = "\t", index_col = False)
    df_gesamt = pd.concat([df_gesamt, df], ignore_index = True)

df_gesamt = df_gesamt.drop(columns = ["Unnamed: 0"])
df_gesamt['Levenshtein Item Min'] = df_gesamt['Levenshtein Item Min'].replace(999, 0)
df_gesamt['Levenshtein Phontype Min'] = df_gesamt['Levenshtein Phontype Min'].replace(999, 0)
df_gesamt['Levenshtein Lextype Min'] = df_gesamt['Levenshtein Lextype Min'].replace(999, 0)
df_gesamt['z_score_item'] = df_gesamt['z_score_item'].replace(527.7358915701176, 0)
df_gesamt['z_score_phontype'] = df_gesamt['z_score_phontype'].replace(529.1860287263053, 0)
df_gesamt['z_score_lextype'] = df_gesamt['z_score_lextype'].replace(529.091394870162, 0)
df_gesamt["Koordinaten"] = df_gesamt["Longitude"].astype(str) + ", " + df_gesamt["Latitude"].astype(str)

In [3]:
df_final = df_gesamt.groupby("Koordinaten").sum()

df_final = df_final.drop(columns = ["Ort", "Item Maurer", "Item DWA", "Phontype Maurer", "Phontype DWA", "Lextype Maurer", "Lextype DWA"])

maurer = []
for i in df_final["Fragebogen-Nr. Maurer"]:
    length = len(i)
    part = int(length/14)
    maurer.append(i[0:part])

df_final["Fragebogen-Nr. Maurer"] = maurer

dwa = []
for j in df_final["Fragebogen-Nr. DWA"]:
    length = len(j)
    part = int(length/14)
    dwa.append(j[0:part])

df_final["Fragebogen-Nr. DWA"] = dwa

long = []
lat = []
for x in df_final.index:
    split = x.split(", ")
    lo = float(split[0])
    la = float(split[1])
    long.append(lo)
    lat.append(la)

df_final["Longitude"] = long
df_final["Latitude"] = lat


In [4]:
# speichern
df_final.to_csv("/home/kopatsch/Masterarbeit/MA/masterarbeit/data/levenshtein_gesamt/levenshtein_gesamt.csv", sep = "\t")